# Personal Financial Report
>version 3.1

this jupyter notebook allows me to keep track of my finances.

# SET UP

In [1]:
## import the Libraries

# standard Libraries
import os
import re
import time
import json5 as json 
from datetime import datetime, timedelta
import pandas as pd
import numpy as np

import functions as f

# used for displaying data and stuff
from IPython.display import display, HTML, FileLink, FileLinks

# used for charts and Graphs
# import plotly
import plotly.express as px
import seaborn as sns
import calmap
from plotly_calplot import calplot

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

# plotly Theme
plotly_theme = 'plotly_dark'

# disable plotly warnings
import warnings
warnings.filterwarnings("ignore")

## Pandas Options

# show all the columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.width', 1000)

# turn off a warning
pd.options.mode.chained_assignment = None  # default='warn'

## Variables
DIR = os.getcwd()
data_DIR = os.path.join(DIR,'data')
old_data_DIR = os.path.join(DIR,'old_data')

if os.path.exists(data_DIR) == False:
    os.mkdir(data_DIR)


In [2]:
this_year = int(datetime.now().strftime("%Y"))
this_month = int(datetime.now().strftime("%m"))

this_year = 2025

# incorporated new data

In [3]:
# merges the old data with the new data...
# and opens the file so the user can fill in the spots that are not automatically filled


# catt = f.get_category_table(f.load_last_x_years(1))
# catt.to_csv(os.path.join(data_DIR,'category_table.csv'), index=False)

data = f.incorporated_data(autofillcat= True, year=this_year,)

#display(data.head(20))
#display(data.tail(20))

f.open_year_in_excel(this_year)


# create dataframes

df_year  
df_90days  
df_30days  
df_07days  
~~df_all~~  


In [4]:
# df_year = f.load_last_x_years(2)
df_year = f.load_year(this_year)
# df_year = f.load_year(2023)
# df_year = f.load_year(2022)

df_year.Date = pd.to_datetime(df_year.Date)

ninety_days = ( df_year.Date.max() - timedelta(days=90)) 
df_90days = df_year[ df_year['Date'] >= ninety_days]
# display(df_90days)

thirty_days = ( df_year.Date.max() - timedelta(days=30)) 
df_30days = df_year[ df_year['Date'] >= thirty_days]
# display(df_30days)

seven_days = ( df_year.Date.max() - timedelta(days=7)) 
df_07days = df_year[ df_year['Date'] >= seven_days]
# display(df_07days)

# df_all = f.get_all_data()


In [5]:
display(HTML('<h2>Date Range</h2>'))
display(HTML(f'<p>{df_year["Date"].min()} to {df_year["Date"].max()}</p>'))


# 🍥 Rolling Weekly

In [6]:
def RollingWeek():
    

    temp = df_year.copy() 
    temp = temp[['YYYY.W','Category','Delta']]
    temp = temp[temp['Delta'] < 0.0]
    temp = temp[temp['Category'] == 'fun']

    df_rw = temp.groupby(['YYYY.W','Category']).sum()
    avg = (df_rw.sum()/len(df_rw))[0]
    df_rw = df_rw.iloc[::-1]
    df_rw = df_rw.reset_index()

    df_rw['avg'] = avg
    df_rw['zero'] = 0
    df_rw['avg-delta'] = (df_rw['avg']*-1.0) + df_rw['Delta']
    df_rw = df_rw[['YYYY.W','Category','avg','Delta','avg-delta']]


    # df_rw = df_rw.melt(id_vars='YYYY.W', value_vars=['avg-delta','zero'], 
    #                     var_name='Category', value_name='Value')
    # print(avg)
    # display(df_rw.head(20))

    display(HTML('<h1>Rolling Week </h1>' 
                  + '<p>Money Spent on fun, groupped by week</p>' 
                  + '<p>Average: {0} </p>'.format(avg) 
                  + '<p>Goal: keep avg-delta above Zero</p>'
                 ))
    
    cm = sns.color_palette('RdYlGn', as_cmap=True)

    display(df_rw.head(10).style.background_gradient(cmap=cm).format(precision=2,thousands=','))


RollingWeek()

,YYYY.W,Category,avg,Delta,avg-delta
0,"2,025.52",fun,-262.75,-35.31,227.44
1,"2,025.51",fun,-262.75,-118.08,144.67
2,"2,025.50",fun,-262.75,-194.42,68.33
3,"2,025.49",fun,-262.75,-227.65,35.10
4,"2,025.48",fun,-262.75,-75.91,186.84
5,"2,025.47",fun,-262.75,-187.17,75.58
6,"2,025.46",fun,-262.75,-231.01,31.74
7,"2,025.45",fun,-262.75,-189.66,73.09
8,"2,025.44",fun,-262.75,-372.97,-110.22
9,"2,025.43",fun,-262.75,-255.07,7.68


# 🌞 SunBurst

In [7]:
# print(*px.colors.named_colorscales())

In [8]:
def make_sunburst(df,title):
    display(HTML('<h2> 🌞 SunBurst: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))

    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(temp['Delta'].sum())))

    fig = px.sunburst(
        temp,
        # path=['YYYYMM','Category','Location'],
        path=['Category','Location'],
        values='Delta',
        color='Delta', 
        hover_data=['Category','Location','Delta','YYYYMMDD'],
        color_continuous_scale='rdylgn_r',
        color_continuous_midpoint=np.average(temp['Delta'], weights=temp['Delta']),
        template = plotly_theme,
        height=600
    )
    fig.show()


In [9]:
# make_sunburst(df_07days,'7 days')
# make_sunburst(df_30days,'30 days')
# make_sunburst(df_90days,'90 days')
make_sunburst(df_year,'Year')

# 🔥 HeatMaps

In [10]:
## run this for a list of the colorscales
# print(*px.colors.named_colorscales())

def heatmap_time_cat(df, time_column, max_value = None, title = '',colorscale = 'RdYlGn'):
    """
    creates a heatmap with time and category 
    df: dataframe
    time_column: the column to use for the columns
    max_value: the value that will reprement the max color in the color scale 
        * min_value = max_value * -1
    title: the title to display 
    colorscale: the columns to use
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp = df.copy()
    temp[time_column] = temp[time_column].astype(float).astype(str)
    
    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = 'Category',
        columns = time_column,
        aggfunc= {'Delta':sum,}
        )
    
    temp = temp.fillna(0)
    temp = temp.reset_index()
    temp = temp[temp.columns.tolist()]
    
    if 'nan' in temp.columns.tolist():
        temp = temp.drop(columns=['nan'])
    
    cols = temp.columns.tolist()
    cols = [c for c in cols if c != 'Category']
    
    
    # Sum of Rows
    temp['Σ'] = temp[cols].sum(axis=1)

    # sort by row sum
    temp = temp.sort_values(by='Σ',ascending=False)
    
    # Sum of Cols
    sums = {}
    absminmax = 0
    for c in temp.columns:
        if c == 'Category':
            sums['Category'] = 'GrandTotal'
        else:
            sums[c] = temp[c].sum()
            absminmax = max([absminmax,abs(sums[c])])
    # print(sums)
    sums = pd.DataFrame(sums,index=[99])

    temp = pd.concat([temp,sums])
    temp = temp.reset_index(drop=True)
    
    cm = sns.color_palette(colorscale, as_cmap=True)
    
    if max_value == None:
        display(temp.style.background_gradient(cmap=cm, vmin=absminmax*-1, vmax=absminmax).format(precision=2,thousands=','))
    else:
        display(temp.style.background_gradient(cmap=cm, vmin=max_value*-1, vmax=max_value).format(precision=2,thousands=','))
    

def heatmap2_year_view(df, title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2'))
    
    temp = df.copy()

    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = 'Date',
        aggfunc= {'Delta':sum,}
        )
    temp = temp.reset_index(drop=False)
    
    events = pd.Series(  list(temp['Delta']) , list(temp['Date']) )
    calmap.calendarplot(events,cmap='RdYlGn', daylabels='MTWTFSS')
    
def heatmap3_year_view(df,title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2>'))
    
    temp = df.copy()
    
    # get the day of the week
    temp['DOW_'] = pd.to_datetime(temp['Date']).dt.strftime("%w")
    temp['DOW'] = pd.to_datetime(temp['Date']).dt.strftime("%a")
    temp['M.W'] = pd.to_datetime(temp['Date']).dt.strftime("%m.%W")

    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = ['DOW_','DOW'],
        columns = 'M.W',
        aggfunc= {
            'Delta':sum,
            }
        )
    
    temp = temp.reset_index()
    temp = temp.drop(columns=['DOW_'])
    temp.set_index("DOW", inplace=True)
    
    temp = temp.fillna(0)
    
    fig = px.imshow(
        temp, 
        text_auto=True, 
        # aspect="auto",
        color_continuous_scale='rdylgn',
        color_continuous_midpoint = 0.0,
        template = plotly_theme,
        height=500,
        )
    fig.update_xaxes(side="top")
    fig.show()

def heatmap4_year_view(df,title = ''):
    """
    creates a heatmap with a year view
    df: dataframe
    title: the title to display 

    --this doesn't work as well as the other one
    """
    display(HTML('<h2> 🔥 HeatMap: ' + title + '</h2>'))

    temp = df.copy()
    temp = temp.fillna(0.0)

    fig = calplot(
        temp,
        x="Date",
        y="Delta",
        dark_theme=True,
        colorscale="rdylgn",
        total_height=600
    )
    fig.show()

    

In [11]:
# heatmap_time_cat(df_07days,time_column='YYYYMMDD',max_value=500,title='7 days')
# heatmap_time_cat(df_07days,time_column='YYYYMMDD',max_value=1000,title='7 days')

In [12]:
# heatmap_time_cat(df_30days,time_column='YYYY.W',max_value=500,title='30 days')
# heatmap_time_cat(df_30days,time_column='YYYY.W',max_value=1000,title='30 days')

In [13]:
# heatmap_time_cat(df_90days,time_column='YYYYMM',max_value=500,title='90 days')
# heatmap_time_cat(df_90days,time_column='YYYYMM',max_value=1000,title='90 days')

In [14]:
# heatmap_time_cat(df_year,time_column='YYYYMM',max_value=500,title='Year')
heatmap_time_cat(df_year,time_column='YYYYMM',max_value=1000,title='Year')

,Category,202501.0,202502.0,202503.0,202504.0,202505.0,202506.0,202507.0,202508.0,202509.0,202510.0,202511.0,202512.0,Σ
0,check,"6,496.96","10,664.73","7,154.44","7,448.60","6,904.60","6,890.17","6,890.74","6,390.14","6,870.30","3,425.06","6,935.75","5,877.93","81,949.42"
1,transfer,0.00,0.00,0.00,"10,000.00","15,000.00",0.00,-41.46,"1,000.00",0.00,"5,000.00",0.00,0.00,"30,958.54"
2,rent,"1,125.00",954.00,"1,135.00","1,173.00","1,208.00","1,152.00","1,151.00",980.00,"1,204.00","1,082.00","1,258.00","1,077.00","13,499.00"
3,tax,0.00,0.00,"13,332.00","1,420.00",0.00,0.00,0.00,0.00,"-3,293.68",0.00,0.00,0.00,"11,458.32"
4,bro,0.00,100.00,100.00,100.00,100.00,200.00,0.00,200.00,0.00,0.00,0.00,0.00,800.00
5,interest,0.21,0.21,0.32,0.00,0.63,1.10,0.00,0.00,0.00,0.00,0.00,0.00,2.47
6,withdraw,0.00,-200.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-200.00
7,donate,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-260.00,0.00,0.00,0.00,-260.00
8,clothes,0.00,0.00,-76.16,0.00,0.00,0.00,0.00,-223.08,0.00,0.00,0.00,0.00,-299.24
9,misc,-4.13,0.00,-67.69,-6.45,-42.11,-58.69,-306.80,-186.10,0.88,0.35,-32.25,0.28,-702.71


In [32]:
# not working with pandas 2.0.3
# heatmap2_year_view(df_year,title='Year_alt1')

In [16]:
heatmap3_year_view(df_year,'Year_alt2')

# 🔥🗓️ HeatCalendar

In [17]:
import calendar

cmap = plt.get_cmap('RdYlGn')

# get the largest absolute delta per day
temp = pd.pivot_table(
    df_year,
    values = 'Delta',
    index = 'YYYYMMDD',
    aggfunc= {'Delta':sum,}
    )
absmax = max(abs(temp['Delta'].min()) ,temp['Delta'].max())
absmax *= 0.5


for mon_num in range(1,13):

    m = calendar.month(this_year,mon_num)

    # print(m)

    mon_table = ''
    for index,l in enumerate(m.split('\n')):
        if index == 0:
            mon_table += f'<tr><td>{this_year}</td><td>{calendar.month_name[mon_num]}</td></tr>'
        elif index == 1:
            mon_table += f'<tr><td>Mo </td><td>Tu </td><td>We </td><td>Th </td><td>Fr </td><td>Sa </td><td>Su </td></tr>'
        else:

            wstr = ''
            for i in range(0,len(l),3):
                d_sum = 0
                
                try:
                    year_month_day = this_year * 10000 + mon_num * 100 + int(l[i:i+2].replace(' ', ''))
                    d_sum = df_year.query(f'YYYYMMDD == {year_month_day}')['Delta'].sum()
                except:
                    pass

                normalized_value = (d_sum + absmax) / (absmax*2)
                normalized_value = max(0.0, min(normalized_value, 1.0))
                # print(normalized_value)
                color = cmap( normalized_value )
                # print(color)
                r, g, b, _ = color
                

                d = l[i:i+2].replace(' ',' ')+ ''
                wstr += f'<td style="height:1rem; width: 3rem; background-color:rgb({int(r*255)},{int(g*255)},{int(b*255)}); color:black"><b>{d}<br/>{int(d_sum)}</b></td>'
            mon_table += '<tr>' + wstr + '</tr>'
    # print(mon_table)
    display(HTML('<table>' + mon_table + '</table>'))


# 📊 Bar-Chart

In [18]:
def make_barchart(df,time_column,title):
    display(HTML('<h2> 📊 Bar-Chart: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(temp['Delta'].sum())))

    temp[time_column] = temp[time_column].astype(float).astype(str)
    
    temp = pd.pivot_table(
        temp,
        values = 'Delta',
        index = [time_column,'Category'],
        aggfunc= {'Delta':sum,}
        )

    temp = temp.reset_index(drop=False)
    temp[time_column] = temp[time_column].astype(float,errors='ignore')
    # temp = temp.sort_values(by = time_column,ascending=True)

    fig = px.bar(
        temp,
        y='Delta',
        x=time_column,
        color='Category',
        hover_data=[time_column,'Category','Delta'],
        template = plotly_theme,
        # text_auto=True,
        text = 'Category',
        height=600,
        )
    
    fig.show()
    


In [19]:

# make_barchart(df=df_07days,time_column='YYYYMMDD',title='7 days')
# make_barchart(df=df_30days,time_column='YYYY.W',title='30 days')
# make_barchart(df=df_90days,time_column='YYYYMM',title='90 Days')
make_barchart(df=df_year,time_column='YYYYMM',title='Year')

In [20]:
# print(*list(df_year.Category.drop_duplicates()),sep='\n')

In [21]:
categories = [
    'fun',
    # 'grocerys',
    # 'amazon',
    # 'transfer',
    # 'pay_off_debt',
    # 'utility',
    # 'misc',
    # 'music',
    # 'health',
    # 'rent',
    # 'transportation',
    # 'phone',
    # 'dating',
    # 'check',
    # 'Costco',
    # 'gift',
    # 'invest',
    # 'cargas',
    # 'homeimprovement',
    # 'mortgage',
    # 'insurance',
    # 'cabin',
    # 'to_cabin',
    # 'tech',
    # 'car',
    # 'taxes',
    # 'interest',
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)

In [22]:
categories = [
    # 'fun',
    # 'grocerys',
    # 'amazon',
    # 'transfer',
    'pay_off_debt',
    # 'utility',
    # 'misc',
    'music',
    'health',
    # 'rent',
    'transportation',
    # 'phone',
    'dating',
    # 'check',
    # 'Costco',
    # 'gift',
    'invest',
    'cargas',
    'homeimprovement',
    'mortgage',
    # 'insurance',
    'cabin',
    'to_cabin',
    'tech',
    'car',
    # 'taxes',
    # 'interest',
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)


In [23]:
categories = [
    # 'fun',
    # 'grocerys',
    # 'amazon',
    # 'transfer',
    # 'pay_off_debt',
    'utility',
    # 'misc',
    # 'music',
    # 'health',
    # 'rent',
    # 'transportation',
    # 'phone',
    # 'dating',
    # 'check',
    # 'Costco',
    # 'gift',
    # 'invest',
    # 'cargas',
    # 'homeimprovement',
    # 'mortgage',
    # 'insurance',
    # 'cabin',
    # 'to_cabin',
    # 'tech',
    # 'car',
    # 'taxes',
    # 'interest',
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)


In [24]:
categories = [
    # 'fun',
    # 'grocerys',
    # 'amazon',
    # 'transfer',
    # 'pay_off_debt',
    # 'utility',
    # 'misc',
    # 'music',
    'health',
    # 'rent',
    # 'transportation',
    # 'phone',
    # 'dating',
    # 'check',
    # 'Costco',
    # 'gift',
    # 'invest',
    # 'cargas',
    # 'homeimprovement',
    # 'mortgage',
    # 'insurance',
    # 'cabin',
    # 'to_cabin',
    # 'tech',
    # 'car',
    # 'taxes',
    # 'interest',
]


mask = df_year['Category'].isin(categories)
temp = df_year[mask]
title = f'Year - filtered categories ({categories})'
make_barchart(df=temp,time_column='YYYYMM',title=title)

In [25]:
try:
    Locations = [
        'Amazon_detail'
    ]

    mask = df_year['Location'].isin(Locations)
    temp = df_year[mask]
    title = f'Year - filtered Location ({Locations})'
    make_barchart(df=temp,time_column='YYYYMM',title=title)
except Exception as e:
    print(e)

Value of 'y' is not the name of a column in 'data_frame'. Expected one of ['YYYYMM', 'Category'] but received: Delta


# 📈 Line-Graph

In [26]:
def make_linegraph(df,title):
    display(HTML('<h2> 📈 Line-Graph: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    df = df.sort_values(by = 'Date',ascending=False)

    fig = px.line(
        df,
        y='Balance',
        x='Date',
        hover_data=['Date','Category','Location','Delta'],
        template = plotly_theme,
        height=600
    )
    fig.show()

In [27]:
# make_linegraph(df_07days,'7 days')
# make_linegraph(df_30days,'30 days')
# make_linegraph(df_90days,'90 days')
make_linegraph(df_year,'Year')

# 📈 Scatter-Graph

In [28]:
def make_scattergraph(df,title):
    display(HTML('<h2> 📈 Scatter-Graph: ' + title + '</h2>'))
    display(HTML('from {0}'.format(df.Date.min())))
    display(HTML('to {0}'.format(df.Date.max())))
    # display(HTML('total {0}'.format(df['Delta'].sum())))

    temp = df.copy() # for 3 weeks
    temp = temp[temp['Delta'] < 0.0]
    temp['Delta'] = abs(temp['Delta'])
    temp = temp[temp['Delta'] != 0.0]

    display(HTML('total {0}'.format(df['Delta'].sum())))
    
    temp['size'] = 0.0
    temp['size'] = temp['Delta']
    # temp.loc[temp['size'] >= 1000,'size'] = 1000
    # temp.loc[temp['size'] < 1,'size'] = 1
    

    # display(temp)
    
    fig = px.scatter(
        temp,
        y='Balance',
        x='Date',
        size='size',
        color='Category',
        # color='Delta',
        hover_data=['Date','Category','Location','Delta'],
        template = plotly_theme,
        height=1000,
        size_max=100,
        opacity=0.80,
    )
    
    fig.update_traces(
        marker=dict(
        # size=12,
        line=dict(
            width=1,
            color='white',
            )
        ),
        selector=dict(mode='markers')
    )
    
    fig.show()

In [29]:
# make_scattergraph(df_07days,'7 days')
# make_scattergraph(df_30days,'30 days')
# make_scattergraph(df_90days,'90 days')
make_scattergraph(df_year,'Year')

# Write Utility Report


In [30]:

def add_one_month(yyyymm):
    # Convert to string and extract year and month
    yyyymm_str = str(yyyymm)
    year = int(yyyymm_str[:4])
    month = int(yyyymm_str[4:])

    # Add one month
    month += 1
    if month > 12:  # If month exceeds December, increment year and reset month
        year += 1
        month = 1

    # Convert back to YYYYMM format
    return int(year * 100 + month)

def date_to_rent(date:int):
    '''
    converts the date to rent
    '''

    result = 1000.0

    rents = {
        0:1000.0,
        20231000:1200.0,
        20240300:1000.0,
    }

    for k in rents.keys():
        if date >= k:
            result = rents[k]
    
    return result


def date_to_thanks(date:int):
    '''
    converts the date to rent
    '''
    result = 0.0

    thanks = {
        0:0,
        # 20250800:200.0
    }

    for k in thanks.keys():
        if date == k:
            result = thanks[k]
    
    return result


# # tests 
# print(date_to_rent(20230900))
# print(date_to_rent(20231000))
# print(date_to_rent(20231100))
# print(date_to_rent(20240300))

def create_utility_report(df, start_yyyymmdd, save_path):
    temp = df[(df['Category'] == 'utility') | (df['Location'].str.contains(r'^(DUKEENERGY|CHARLOTTE DES:|AT&T \*PAYMENT|PIEDMONT)', regex=True))]
    
    temp = temp[temp.YYYYMMDD >= start_yyyymmdd]
    temp = temp[['Date','YYYYMM','Location','Delta']]
    # temp = temp[['YYYYMM','Location','Delta']]

    # # override ATT in order to exclude the phone bill
    # att = temp['Location'].str.startswith('ATT DES')
    # temp.loc[att, 'Delta'] = -80.0

    # temp['Location'] = temp['Location'].str[0:20]

    temp['HalfDelta'] = temp['Delta']/2.0
    
    YYYYMM_list = list(temp.YYYYMM.drop_duplicates())
    
    for YYYYMM in YYYYMM_list:
        try:
            dfyyyymm = temp[temp.YYYYMM == YYYYMM]
            
            space = pd.DataFrame({},index=[0])
            dfyyyymm = pd.concat([dfyyyymm,space])

            d_sum = dfyyyymm.copy().Delta.sum()
            hd_sum = dfyyyymm.copy().HalfDelta.sum()

            YYYYMMpOne = add_one_month(YYYYMM)
            
            # add sum row
            sum = {
                'Date': YYYYMM,
                'YYYYMM': YYYYMM,
                'Location': 'sum-of-utilities',
                'Delta': d_sum,
                'HalfDelta': hd_sum,
                }
            sum = pd.DataFrame(sum,index=[0])
            dfyyyymm = pd.concat([dfyyyymm,sum])

            rent_row = {
                'Date': YYYYMMpOne,
                'YYYYMM': YYYYMMpOne,
                'Location': 'rent',
                'Delta': '...',
                'HalfDelta': -1* date_to_rent(int(datetime.now().strftime("%Y%m%d"))),
                }
            rent_row = pd.DataFrame(rent_row,index=[0])
            dfyyyymm = pd.concat([dfyyyymm,rent_row])


            thank_value = date_to_thanks(int(datetime.now().strftime("%Y%m00")))

            if (thank_value > 0.0):
                thank_row = {
                    'Date': YYYYMMpOne,
                    'YYYYMM': YYYYMMpOne,
                    'Location': 'Thanks 👍',
                    'Delta': '...',
                    'HalfDelta': '+' + str(thank_value),
                    }
                thank_row = pd.DataFrame(thank_row,index=[0])
                dfyyyymm = pd.concat([dfyyyymm,thank_row])

            total_row = {
                'Date': YYYYMMpOne,
                'YYYYMM': YYYYMMpOne,
                'Location': 'total (sum-of-utilities + rent)',
                'Delta': "N/A",
                'HalfDelta': -date_to_rent(int(datetime.now().strftime("%Y%m%d"))) + hd_sum + thank_value,
                }
            total_row = pd.DataFrame(total_row,index=[0])
            dfyyyymm = pd.concat([dfyyyymm,total_row])

     
            extention = '.csv'
            file = os.path.join(save_path, 'RAU_' + str(int(YYYYMMpOne)) + extention)
            dfyyyymm.to_csv(file,index=False)
            # if verbose:

            # print(file)
            
            display(HTML('<h2> saved : ' + file + '</h2>'))
            # display(HTML('<a href="' + 'file:/'+ file.replace('/','\\') + '"> saved : ' + 'utilities_' + str(int(YYYYMM)) + '.csv' + '</a>'))
            display(FileLink(file))
            # print(f'[file]({file})')
            

        except Exception as e:
            print(str(e))

    
create_utility_report(df_year,20251100,r'C:\Users\JGarza\Google Drive\HouseStuff\rent_and_utilities')

# test
# create_utility_report(f.load_year(2022),202206,r'C:\Users\JGarza\Google Drive\HouseStuff\Utilities')

C:\Users\JGarza\Google Drive\HouseStuff\rent_and_utilities\RAU_202601.csv

C:\Users\JGarza\Google Drive\HouseStuff\rent_and_utilities\RAU_202512.csv